### Spark Session

In [5]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.ui.port", "4042")
    #.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0,io.delta:delta-spark_2.12:3.3.2")
    #.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    #.set("spark.executor.heartbeatInterval", "300000")
    #.set("spark.network.timeout", "400000")
    # Minio specific settings
    #.set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    #.set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    #.set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    #.set("spark.hadoop.fs.s3a.path.style.access", "true")
    #.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.sql.warehouse.dir", "s3a://defaultfs/spark/warehouse")
    #.set("spark.hadoop.fs.defaultFS", "s3a://defaultfs/")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-sql-notebook').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

25/10/11 20:38:13 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/10/11 20:38:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/11 20:38:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
#
states = {"NY":"New York", "CA":"California", "FL":"Florida", "TX":"Texas", "CH":"Chicago", "OH":"Ohio", "NJ":"New Jersey", "UT":"Utah" }

#
#
#
sample_data = [
    (("James", "", "Smith"), "36636", "Sales", 34, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 3100.50, "OH", "07-01-2019 12 01 19 406"),
    (("Michael", "Rose", ""), "40288", "Finance", 25, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'brown', 'eye': 'brown'}, "M", 4300.65, "CA", "06-24-2019 12 01 19 406"),
    (("Robert", "", "Williams"), "42114", "Marketing", 27, "1992-06-23", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'black'}, "M", 1400.80, "NJ", "11-16-2019 16 50 59 406"),
    (("Maria", "Anne", "Jones"), "39192", "Finance", 19, "1987-05-05", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 5500.75, "NY", "11-16-2019 16 50 59 406"),
    (("Jen", "Mary", "Brown"), "39191", "Marketing", 24, "1990-01-01", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 0.00, "UT", "11-16-2019 16 50 59 406"),
    (("Jeff", "A", "Jones"), "39193", "Finance", 19, "2000-07-17", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 5500.75, "CA", "11-16-2019 16 50 59 406"),
    (("Robert", "", "Williams"), "42114", "Marketing", 27, "1992-06-23", "false", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 1400.80, "NJ", "11-16-2019 16 50 59 406"),
    (("James", "", "Smith"), "36636", "Sales", 34, "2006-01-01", "true", ["Java", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "M", 3100.50, "OH", "07-01-2019 12 01 19 406"),
    (("Scott", "M", "Brown"), "39194", "Marketing", 24, "1994-10-01", "false", ["Python", "Scala", "C++"], {'hair': 'black', 'eye': 'brown'}, "F", 0.00, None, "11-16-2019 16 50 59 406")
]

#
#
#
sample_schema = StructType([
    StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True)
    ])),
    StructField('id', StringType(), True),
    StructField('department', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('join_date', StringType(), True),
    StructField('isactive', StringType(), True),
    StructField('languages', ArrayType(StringType(), True), True),
    StructField('properties', MapType(StringType(),StringType()), True),
    StructField('gender', StringType(), True),
    StructField('salary', FloatType(), True),
    StructField('state', StringType(), True),
    StructField('timestamp', StringType(), True)
])

In [7]:
#
df = spark.createDataFrame(data = sample_data, schema = sample_schema)

#
df.printSchema()

#
df.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- join_date: string (nullable = true)
 |-- isactive: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- state: string (nullable = true)
 |-- timestamp: string (nullable = true)



+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|name                |id   |department|age|join_date |isactive|languages           |properties                   |gender|salary |state|timestamp              |
+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|{James, , Smith}    |36636|Sales     |34 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> black}|M     |3100.5 |OH   |07-01-2019 12 01 19 406|
|{Michael, Rose, }   |40288|Finance   |25 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> brown}|M     |4300.65|CA   |06-24-2019 12 01 19 406|
|{Robert, , Williams}|42114|Marketing |27 |1992-06-23|false   |[Java, Scala, C++]  |{eye -> black, hair -> black}|M     |1400.8 |NJ   |11-16-2019 16 50 59 406|
|{Maria, Anne, Jones}|39192|Finance   |1

#### Date Columns

In [8]:
df.show(n=20, truncate=True, vertical=False)

+--------------------+-----+----------+---+----------+--------+--------------------+--------------------+------+-------+-----+--------------------+
|                name|   id|department|age| join_date|isactive|           languages|          properties|gender| salary|state|           timestamp|
+--------------------+-----+----------+---+----------+--------+--------------------+--------------------+------+-------+-----+--------------------+
|    {James, , Smith}|36636|     Sales| 34|2006-01-01|    true|  [Java, Scala, C++]|{eye -> brown, ha...|     M| 3100.5|   OH|07-01-2019 12 01 ...|
|   {Michael, Rose, }|40288|   Finance| 25|2006-01-01|    true|  [Java, Scala, C++]|{eye -> brown, ha...|     M|4300.65|   CA|06-24-2019 12 01 ...|
|{Robert, , Williams}|42114| Marketing| 27|1992-06-23|   false|  [Java, Scala, C++]|{eye -> black, ha...|     M| 1400.8|   NJ|11-16-2019 16 50 ...|
|{Maria, Anne, Jones}|39192|   Finance| 19|1987-05-05|   false|  [Java, Scala, C++]|{eye -> brown, ha...|     F|

In [9]:
df.show(2)

+-----------------+-----+----------+---+----------+--------+------------------+--------------------+------+-------+-----+--------------------+
|             name|   id|department|age| join_date|isactive|         languages|          properties|gender| salary|state|           timestamp|
+-----------------+-----+----------+---+----------+--------+------------------+--------------------+------+-------+-----+--------------------+
| {James, , Smith}|36636|     Sales| 34|2006-01-01|    true|[Java, Scala, C++]|{eye -> brown, ha...|     M| 3100.5|   OH|07-01-2019 12 01 ...|
|{Michael, Rose, }|40288|   Finance| 25|2006-01-01|    true|[Java, Scala, C++]|{eye -> brown, ha...|     M|4300.65|   CA|06-24-2019 12 01 ...|
+-----------------+-----+----------+---+----------+--------+------------------+--------------------+------+-------+-----+--------------------+
only showing top 2 rows



In [10]:
df.show(4,False)

+--------------------+-----+----------+---+----------+--------+------------------+-----------------------------+------+-------+-----+-----------------------+
|name                |id   |department|age|join_date |isactive|languages         |properties                   |gender|salary |state|timestamp              |
+--------------------+-----+----------+---+----------+--------+------------------+-----------------------------+------+-------+-----+-----------------------+
|{James, , Smith}    |36636|Sales     |34 |2006-01-01|true    |[Java, Scala, C++]|{eye -> brown, hair -> black}|M     |3100.5 |OH   |07-01-2019 12 01 19 406|
|{Michael, Rose, }   |40288|Finance   |25 |2006-01-01|true    |[Java, Scala, C++]|{eye -> brown, hair -> brown}|M     |4300.65|CA   |06-24-2019 12 01 19 406|
|{Robert, , Williams}|42114|Marketing |27 |1992-06-23|false   |[Java, Scala, C++]|{eye -> black, hair -> black}|M     |1400.8 |NJ   |11-16-2019 16 50 59 406|
|{Maria, Anne, Jones}|39192|Finance   |19 |1987-05-0

In [11]:
df.show(truncate=False)

+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|name                |id   |department|age|join_date |isactive|languages           |properties                   |gender|salary |state|timestamp              |
+--------------------+-----+----------+---+----------+--------+--------------------+-----------------------------+------+-------+-----+-----------------------+
|{James, , Smith}    |36636|Sales     |34 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> black}|M     |3100.5 |OH   |07-01-2019 12 01 19 406|
|{Michael, Rose, }   |40288|Finance   |25 |2006-01-01|true    |[Java, Scala, C++]  |{eye -> brown, hair -> brown}|M     |4300.65|CA   |06-24-2019 12 01 19 406|
|{Robert, , Williams}|42114|Marketing |27 |1992-06-23|false   |[Java, Scala, C++]  |{eye -> black, hair -> black}|M     |1400.8 |NJ   |11-16-2019 16 50 59 406|
|{Maria, Anne, Jones}|39192|Finance   |1

In [12]:
df.limit(1).show()

+----------------+-----+----------+---+----------+--------+------------------+--------------------+------+------+-----+--------------------+
|            name|   id|department|age| join_date|isactive|         languages|          properties|gender|salary|state|           timestamp|
+----------------+-----+----------+---+----------+--------+------------------+--------------------+------+------+-----+--------------------+
|{James, , Smith}|36636|     Sales| 34|2006-01-01|    true|[Java, Scala, C++]|{eye -> brown, ha...|     M|3100.5|   OH|07-01-2019 12 01 ...|
+----------------+-----+----------+---+----------+--------+------------------+--------------------+------+------+-----+--------------------+



In [13]:
df.show(df.count())

+--------------------+-----+----------+---+----------+--------+--------------------+--------------------+------+-------+-----+--------------------+
|                name|   id|department|age| join_date|isactive|           languages|          properties|gender| salary|state|           timestamp|
+--------------------+-----+----------+---+----------+--------+--------------------+--------------------+------+-------+-----+--------------------+
|    {James, , Smith}|36636|     Sales| 34|2006-01-01|    true|  [Java, Scala, C++]|{eye -> brown, ha...|     M| 3100.5|   OH|07-01-2019 12 01 ...|
|   {Michael, Rose, }|40288|   Finance| 25|2006-01-01|    true|  [Java, Scala, C++]|{eye -> brown, ha...|     M|4300.65|   CA|06-24-2019 12 01 ...|
|{Robert, , Williams}|42114| Marketing| 27|1992-06-23|   false|  [Java, Scala, C++]|{eye -> black, ha...|     M| 1400.8|   NJ|11-16-2019 16 50 ...|
|{Maria, Anne, Jones}|39192|   Finance| 19|1987-05-05|   false|  [Java, Scala, C++]|{eye -> brown, ha...|     F|

In [14]:
df.show(2,vertical=True)

-RECORD 0--------------------------
 name       | {James, , Smith}     
 id         | 36636                
 department | Sales                
 age        | 34                   
 join_date  | 2006-01-01           
 isactive   | true                 
 languages  | [Java, Scala, C++]   
 properties | {eye -> brown, ha... 
 gender     | M                    
 salary     | 3100.5               
 state      | OH                   
 timestamp  | 07-01-2019 12 01 ... 
-RECORD 1--------------------------
 name       | {Michael, Rose, }    
 id         | 40288                
 department | Finance              
 age        | 25                   
 join_date  | 2006-01-01           
 isactive   | true                 
 languages  | [Java, Scala, C++]   
 properties | {eye -> brown, ha... 
 gender     | M                    
 salary     | 4300.65              
 state      | CA                   
 timestamp  | 06-24-2019 12 01 ... 
only showing top 2 rows



In [15]:
#
#
# 
df.show(2, truncate=False, vertical=True)

-RECORD 0-----------------------------------
 name       | {James, , Smith}              
 id         | 36636                         
 department | Sales                         
 age        | 34                            
 join_date  | 2006-01-01                    
 isactive   | true                          
 languages  | [Java, Scala, C++]            
 properties | {eye -> brown, hair -> black} 
 gender     | M                             
 salary     | 3100.5                        
 state      | OH                            
 timestamp  | 07-01-2019 12 01 19 406       
-RECORD 1-----------------------------------
 name       | {Michael, Rose, }             
 id         | 40288                         
 department | Finance                       
 age        | 25                            
 join_date  | 2006-01-01                    
 isactive   | true                          
 languages  | [Java, Scala, C++]            
 properties | {eye -> brown, hair -> brown} 
 gender   

In [16]:
#
# head() function on a dataframe with 'n' numbered parameter will return that many number of records from the top as list of rows.
#

df.head()


Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406')

In [17]:
#
# head() function on a dataframe with 'n' numbered parameter will return that many number of records from the top as list of rows.
#
df.head(2)

[Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406'),
 Row(name=Row(firstname='Michael', middlename='Rose', lastname=''), id='40288', department='Finance', age=25, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'brown'}, gender='M', salary=4300.64990234375, state='CA', timestamp='06-24-2019 12 01 19 406')]

In [18]:
#
# tail() function on a dataframe with 'n' numbered parameter will return that many number of records from the bottom as list of rows.
#
df.tail(2)

[Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406'),
 Row(name=Row(firstname='Scott', middlename='M', lastname='Brown'), id='39194', department='Marketing', age=24, join_date='1994-10-01', isactive='false', languages=['Python', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='F', salary=0.0, state=None, timestamp='11-16-2019 16 50 59 406')]

In [19]:
#
# first() function on a dataframe will return first row from the dataframe
#
df.first()


Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406')

In [20]:
#
# Using take() function on a dataframe with 'n' numbered parameter will return that many number of records from the top as list of rows.
#
df.take(2)

[Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406'),
 Row(name=Row(firstname='Michael', middlename='Rose', lastname=''), id='40288', department='Finance', age=25, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'brown'}, gender='M', salary=4300.64990234375, state='CA', timestamp='06-24-2019 12 01 19 406')]

In [21]:
# Using collect() function on a dataframe will return all the rows from the dataframe as list of rows
df.collect()

[Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406'),
 Row(name=Row(firstname='Michael', middlename='Rose', lastname=''), id='40288', department='Finance', age=25, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'brown'}, gender='M', salary=4300.64990234375, state='CA', timestamp='06-24-2019 12 01 19 406'),
 Row(name=Row(firstname='Robert', middlename='', lastname='Williams'), id='42114', department='Marketing', age=27, join_date='1992-06-23', isactive='false', languages=['Java', 'Scala', 'C++'], properties={'eye': 'black', 'hair': 'black'}, gender='M', salary=1400.800048828125, state='NJ', timestamp='11-16-2019 16 50 59 406'),
 Row(name=Row(firstname='Maria', middlename='A

In [22]:
# limit() function can be used to limit records before using collect() function on a dataframe. Records are limited to 1 in the below example.
df.limit(1).collect()


[Row(name=Row(firstname='James', middlename='', lastname='Smith'), id='36636', department='Sales', age=34, join_date='2006-01-01', isactive='true', languages=['Java', 'Scala', 'C++'], properties={'eye': 'brown', 'hair': 'black'}, gender='M', salary=3100.5, state='OH', timestamp='07-01-2019 12 01 19 406')]